In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import wandb
import torch
import torch.nn as nn

import utils
from naive_model import NaiveModel
import encoding

In [3]:
NUM_EMBEDDING = 2000

assert torch.cuda.is_available()
torch.backends.cudnn.benchmark = True
torch.set_default_tensor_type('torch.cuda.FloatTensor')

def to_device(d):
    if hasattr(d, 'cuda'):
        return d.cuda()
    return {k: v.cuda() for k, v in d.items()}

class Model(nn.Module):
    def __init__(self, units, combinations=encoding.NAMES):
        super().__init__()
        self.units = units

        self.embed = nn.Embedding(num_embeddings=NUM_EMBEDDING, embedding_dim=units)
        self.lstm = nn.LSTM(input_size=units, hidden_size=units, num_layers=1, batch_first=True, bidirectional=True)

        self.tasks = {}
        for combination in combinations:
            out = nn.Linear(in_features=units, out_features=encoding.class_size(combination))
            self.tasks[combination] = out
            setattr(self, encoding.class_name(combination), out)

    def forward(self, x):
        embeds = self.embed(x)

        lstm_out, (h_n, c_n) = self.lstm(embeds)
        left, right = torch.chunk(h_n, 2, dim=0)
        merge = torch.squeeze(left + right)

        outputs = { combination: f(merge) for combination, f in self.tasks.items() }
        return outputs


In [4]:
def sanity():
    model = to_device(Model(100, combinations=[('B', 'T')]))
    print(model)
    with torch.no_grad():
        verbs = encoding.wordlist2numpy(["אתאקלם", "יכפיל"])
        labels = {'B': torch.Tensor([3, 5]), 'T': torch.Tensor([2, 4])}
        verbs = to_device(torch.from_numpy(verbs).to(torch.int64))
        tag_scores = model(verbs)
        for combination in tag_scores:
            print(combination)
            v = tag_scores[combination]
            print(f'{v=}')
            print(f'{labels=}')
            print()

sanity()

Model(
  (embed): Embedding(2000, 100)
  (lstm): LSTM(100, 100, batch_first=True, bidirectional=True)
  (BxT): Linear(in_features=100, out_features=28, bias=True)
)
('B', 'T')
v=tensor([[-1.1145e-02, -2.1850e-02, -1.0326e-01, -3.6377e-02,  5.9284e-02,
         -3.2209e-03,  4.0498e-02, -1.4735e-01, -6.6154e-02, -2.4741e-02,
         -2.4274e-02, -1.2280e-01,  1.6434e-01,  3.2711e-02, -1.1328e-01,
          7.5939e-02, -1.4824e-01, -1.6729e-01, -1.1893e-01,  7.0057e-02,
         -1.0119e-01,  7.2321e-02, -4.6535e-02,  6.9934e-02, -1.5001e-01,
         -2.7894e-01, -1.1560e-02,  1.5289e-01],
        [-1.1771e-01,  1.1270e-01, -1.1776e-01,  5.1567e-02, -1.5437e-01,
         -1.2678e-01, -2.7054e-02,  3.7978e-03, -1.8110e-04,  2.6344e-02,
         -1.6779e-01, -4.6006e-02,  1.1209e-01,  1.6043e-01,  1.2202e-01,
         -2.8303e-02, -5.5611e-02, -6.9056e-02,  2.2200e-03,  3.4607e-02,
          1.4692e-02,  2.7360e-02,  4.0473e-02,  1.1863e-01, -9.5719e-02,
         -2.6245e-01, -5.5231e-02

In [12]:
BATCH_SIZE = 128

def fit(model, train, test, *, epochs,  runsize, criterion, optimizer, phases, teacher):
    x_train, y_train = utils.batch_xy(train, BATCH_SIZE)
    x_test, y_test = utils.batch_xy(test, BATCH_SIZE)
    x = {'train': x_train, 'test': x_test}
    y = {'train': y_train, 'test': y_test}

    stats = utils.Stats(model.tasks.keys(), runsize)
    
    for epoch in range(epochs):
        stats.epoch_start()
        
        for phase in phases:
            model.train(mode=phase == 'train')

            stats.phase_start(phase, batches_in_phase=len(x[phase]))

            for inputs, labels in zip(x[phase], y[phase]):
                stats.batch_start()

                inputs = to_device(inputs)
                labels = to_device(labels)
                
                with utils.conditional_grad(phase=='train'):
                    outputs = model(inputs)

                if teacher is not None:
                    pseudo_labels = teacher(inputs)
                    losses = {k: criterion(outputs[k].double(), pseudo_labels[k]) for k in outputs}
                else:
                    losses = {combination: criterion(output.double(), labels[combination])
                              for combination, output in outputs.items()}
                
                if phase == 'train':
                    stats.assert_reasonable_initial(losses, nn.CrossEntropyLoss)
                
                loss = sum(losses.values())

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                
                stats.update(loss=loss.item(),
                             batch_size=inputs.size(0),
                             d={k: (outputs[k], labels[k]) for k in outputs})
                
                stats.batch_end()
            stats.phase_end()
        stats.epoch_end()
    return stats

@torch.no_grad()
def predict(model, *verbs):
    model.eval()
    verbs = encoding.wordlist2numpy(verbs)
    verbs = to_device(torch.from_numpy(verbs).to(torch.int64))
    outputs = model(verbs)
    res = {}
    # FIX: assumes no overlaps
    for combination, v in outputs.items():
        combined_index = torch.argmax(v).cpu().data.numpy()
        indices = np.unravel_index(combined_index, encoding.combined_shape(combination))
        if isinstance(combination, str):
            combination = tuple([combination])
        for k, i in zip(combination, indices):
            # assert k not in res, "Overlapping classes are not handled"
            s = k
            if k in res:
                s += "'"
            res[s] = encoding.from_category(k, i)
    if all(r in res for r in ['R1', 'R2', 'R3', 'R4']):
        res['R'] = ''.join(res[k] for k in ['R1', 'R2', 'R3', 'R4']).replace('.', '')
    return res

In [9]:
torch.manual_seed(0)
np.random.seed(0)

arity = '3'
gen = 'all'
artifact_name = f'{gen}_{arity}_shufroot'
filename = f'synthetic/{artifact_name}.tsv'  # all_verbs_shuffled
test_size = 5000

artifact = wandb.Artifact(artifact_name, type='dataset')
artifact.add_file(filename)

(train_x, pre_train_y), (test_x, pre_test_y) = encoding.load_dataset_split(filename, split=test_size)

utils.shuffle_in_unison([train_x, *pre_train_y.values()])

In [7]:
def naive_config(filename):
    return {
        'model': NaiveModel.learn_from_file(filename),
        'phases': ['test'],
        'criterion': nn.CrossEntropyLoss(),
        'optimizer': None
    }

def standard_config(model, lr):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    return {
        'model': model,
        'criterion': nn.CrossEntropyLoss(),
        'optimizer': optimizer,
        'phases': ['train', 'test'],
        'teacher': None
    }

def teacher_config(train, model, lr):
    res = standard_config(model, lr)
    res['teacher'] = NaiveModel.learn_from_data(train)
    res['criterion'] = nn.BCEWithLogitsLoss()  # BCELoss: works, but total loss is nan
    return res

In [14]:
%env WANDB_SILENT true
%env WANDB_MODE run

config = {
    'optimizer': 'adam',
    'batch_size': BATCH_SIZE,
    'epochs': 1,
    'runsize': 2 * 8192 // BATCH_SIZE,
    'test_size': test_size,
}

def experiment(combinations):
    torch.manual_seed(1)
    np.random.seed(1)
    train_y = utils.ravel_multi_index(pre_train_y, combinations)
    test_y = utils.ravel_multi_index(pre_test_y, combinations)
    train = (train_x, train_y)
    test = (test_x, test_y)
    
    units = 400
    lr = 8e-4

    config.update({
        'units': units,
        'lr': lr,
    })
    names_str = '+'.join(encoding.class_name(combination) for combination in combinations)
    run = wandb.init(project="rootem",
                     group=f'joint-{gen}-{arity}-noroot',  # f'lr_units_grid_search-{arity}-{wandb.util.generate_id()}',
                     name=f'joint-{gen}-{arity}-{names_str}-noroot',  # f'{gen}-{arity}-{lr:.0e}',# f'{arity}-batch_{BATCH_SIZE}', # f'all-{arity}-lr_{lr:.0e}-units_{units}',
                     tags=[gen, arity, 'synthetic', 'shuffle-root', 'no_prefix', 'shuffle', 'partitions'],
                     config=config)
    with run:
        run.use_artifact(artifact)

        wandb.config.update(config, allow_val_change=True)

        print(config)

        model = to_device(Model(units=units, combinations=combinations))
        if isinstance(model, nn.Module):
            wandb.watch(model)

        stats = fit(train=train,
            test=test,
            epochs=config['epochs'],
            runsize=config['runsize'],
            **standard_config(model, lr)
        )
        wandb.save(f"simple_{arity}.h5")

    return model, stats

model, stats = experiment(['B', ('P', 'G'), 'R1', 'R2', 'R4'])

env: WANDB_SILENT=true
env: WANDB_MODE=run


{'optimizer': 'adam', 'batch_size': 128, 'epochs': 1, 'runsize': 128, 'test_size': 5000, 'units': 400, 'lr': 0.0008}
 1  5376/ 5470 B_acc: 0.825 PxG_acc: 0.778 R1_acc: 0.972 R2_acc: 0.815 R4_acc: 0.984 P_acc: 0.980 G_acc: 0.788 Loss: 1.2006
 1    39/   39 B_acc: 0.792 PxG_acc: 0.790 R1_acc: 0.920 R2_acc: 0.803 R4_acc: 0.936 P_acc: 0.981 G_acc: 0.801 Loss: 1.8084


In [13]:
def print_words(model):
    print(predict(model, 'הבריל'))
    print(predict(model, 'חגוו'))
    print(predict(model, 'עגו'))
    print(predict(model, 'צירלל'))
    print(predict(model, "השטקרפתי"))
    print(predict(model, "ישסו"))
print_words(model)

{'B': 'הפעיל', 'P': 'יחיד', 'G': 'נקבה', 'R1': 'ב', 'R2': 'ר', 'R4': 'ל'}
{'B': 'פעל', 'P': 'רבים', 'G': 'זכר', 'R1': 'ח', 'R2': 'ג', 'R4': 'ג'}
{'B': 'פעל', 'P': 'רבים', 'G': 'זכר', 'R1': 'ע', 'R2': 'ו', 'R4': 'ג'}
{'B': 'פיעל', 'P': 'יחיד', 'G': 'זכר', 'R1': 'צ', 'R2': 'ר', 'R4': 'ל'}
{'B': 'התפעל', 'P': 'יחיד', 'G': 'נקבה', 'R1': 'ש', 'R2': 'ק', 'R4': 'פ'}
{'B': 'פעל', 'P': 'רבים', 'G': 'זכר', 'R1': 'ש', 'R2': 'ס', 'R4': 'י'}
